In [ ]:
!pip install yfinance
''' We are using yahoo finance dataset because it has a lot of data'''

' We are using yahoo finance dataset because it has a lot of data'

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
#standard data manipulation libraries
from pandas_datareader import data as pdr
#useful for pulling data from yfinance
from sklearn import linear_model
import statsmodels.api as sm
import yfinance as yf

In [ ]:
yf.Ticker("AAPL").history(start = "2023-01-01", end = "2023-01-04")


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2023-01-03 00:00:00-05:00,129.555841,130.17239,123.479803,124.374802,112117500,0.0,0.0


In [ ]:
ticker_list = 'TGT TSLA F GM  MSFT ETSY SPOT NVDA AMD META BABA PYPL NFLX SQ CRM XOM TEAM TWLO V UNH JPM SBUX ADBE BA NIO BAC PANW SNOW COP PDD CVX AVGO FIS EL MRNA QCOM WFC HD INTC OXY ENPH PFE INTU GS CAT NKE DIS TMUS TXN LLY MU CRWD MA NOW LRCX SHOP CVX AMGN JNJ CMCSA MCD DKNG SLB MRK TMO BILL CSCO ROKU HON HUM NET AMAT PG DHR T ADI CI MCHP SCHW PEP LMT WMT C WDAY VZ ACN BMY MRVL DDOG MNST EXP GM ILMN MRO KO IBM PNC ORCL ZS DOW ANET ADSK PLTR FSLR MAR AXP HAL CCL MDB FTNT UAL RTX BIDU VLO AAL UPS RCL HPQ PGR TJX LVS ADP CNC YUM GD OKTA PINS LULU DAL DPZ ATVI SNAP ZM EBAY AEP CL EA SNPS MGM MTCH WB TTWO PAYC LCID DOCU ZI BILI PLUG K COF SPLK CHWY PTON BURL LUV Z LYFT AKAM UBER'.split()
ticker_list = tuple(ticker_list)
#These are a list of stocks that we are looking at

In [ ]:
start = dt.date.today() - dt.timedelta(days = int(365.25*1.5))
mid = dt.date.today() - dt.timedelta(days = int(365.25*0.5))
end = dt.date.today()
#We are doing time series analysis, based on difference between start and end

In [ ]:
print(start, mid, end)

2022-06-06 2023-06-06 2023-12-05


In [ ]:
yf.__version__

'0.2.32'

In [ ]:
dataset = yf.download(list(ticker_list), start=start, end=mid)

[*********************100%%**********************]  157 of 157 completed


In [ ]:
dataset.head()

Adj Close                                                  \
                  AAL         ACN        ADBE         ADI         ADP   
Date                                                                    
2022-06-06  16.299999  297.577759  429.459991  160.689346  212.955551   
2022-06-07  16.690001  299.335480  433.420013  162.322769  215.642853   
2022-06-08  16.150000  295.536835  428.839996  160.436539  213.421219   
2022-06-09  15.470000  288.496033  426.420013  157.130798  210.253281   
2022-06-10  14.710000  279.297089  393.839996  152.570755  206.276306   

                                                                       ...  \
                  ADSK        AEP        AKAM        AMAT         AMD  ...   
Date                                                                   ...   
2022-06-06  208.600006  96.396652  100.650002  113.377174  105.650002  ...   
2022-06-07  207.169998  96.746117  101.769997  114.037727  105.279999  ...   
2022-06-08  203.729996  94.951553  100.760002  110.873016  101.900002  ...   
2022-06-09  195.059998  92.807518   99.820000  105.726669   98.800003  ...   
2022-06-10  187.080002  92.807518   97.440002  100.442314   94.820000  ...   

             Volume                                                           \
                 WB     WDAY       WFC       WMT       XOM      YUM        Z   
Date                                                                           
2022-06-06  1262100  2116800  11867200   6618500  20682100  1175500  4340800   
2022-06-07  1082200  2179700  13281600  15902600  37810200  1047300  3435200   
2022-06-08  1808600  1560800  12364400   7507600  31613200  1164800  3137900   
2022-06-09   869100  1725100  20142400   6626700  26607000  1052700  5564100   
2022-06-10  5650500  3265500  27619300   8412700  30950500  1222900  4473200   

                                       
                 ZI       ZM       ZS  
Date                                   
2022-06-06  4149500  2867500  2028200  
2022-06-07  4800000  3951900  1613900  
2022-06-08  4437800  3583900  2280900  
2022-06-09  3989200  4302600  1968800  
2022-06-10  5611200  5768900  2228200  

[5 rows x 942 columns]

In [ ]:
data = dataset

In [ ]:
dataset = data['Adj Close']
"""the pdr get_data_yahoo function pulls yahoo finance data for all the stocks in ticker_list from start to mid.
the returned columns are Open, High, Low, Adj Close, Close, and Volume."""


stocks_returns = np.log(dataset/dataset.shift(1))
print(stocks_returns)

"""
This accounts for the inherent asymmetry that stocks can only go down 100% but can go up
unlimited amounts. In other words, it rescales [-100, infinity] to [-infinity, infinity]"""
corr_matrix = stocks_returns.corr()
print(corr_matrix)

"""
The .corr() function calculates the correlation between two time series
For this case, we'll take the pairwise correlations between every pair of
stocks and use the highest pairs as candidates for cointegration.
"""

                 AAL       ACN      ADBE       ADI       ADP      ADSK  \
Date                                                                     
2022-06-06       NaN       NaN       NaN       NaN       NaN       NaN   
2022-06-07  0.023645  0.005889  0.009179  0.010114  0.012540 -0.006879   
2022-06-08 -0.032890 -0.012771 -0.010623 -0.011688 -0.010356 -0.016744   
2022-06-09 -0.043017 -0.024112 -0.005659 -0.020820 -0.014955 -0.043488   
2022-06-10 -0.050375 -0.032405 -0.079480 -0.029450 -0.019096 -0.041771   
...              ...       ...       ...       ...       ...       ...   
2023-05-26 -0.002088  0.039780  0.057803  0.023731  0.009837  0.006760   
2023-05-30  0.018640  0.015784  0.004372  0.005085 -0.011169 -0.019598   
2023-05-31  0.010884 -0.008171  0.001389  0.001464 -0.005392  0.022058   
2023-06-01 -0.005427 -0.002356  0.021219  0.015192  0.013403  0.019420   
2023-06-02  0.011498  0.020079  0.022292  0.001391  0.021993  0.004613   

                 AEP      AKAM      A

"\nThe .corr() function calculates the correlation between two time series\nFor this case, we'll take the pairwise correlations between every pair of\nstocks and use the highest pairs as candidates for cointegration.\n"

In [ ]:
processed_corr = corr_matrix.mask(np.tril(np.ones(corr_matrix.shape)).astype(bool))
#Set diagonal elements and lower triangular elements to nans
#Serves to remove duplicates and 1-correlations between an element and itself
s =  processed_corr.unstack()
#Reshapes the correlation matrix into a list of ((row, column), correlation value) pairs
so = s.sort_values(kind="quicksort", ascending = False)
#Sort the pairs by correlation value
display(so[:30])

LRCX  AMAT    0.934561
V     MA      0.905582
SLB   HAL     0.901445
XOM   CVX     0.900646
MRO   COP     0.896298
MCHP  ADI     0.895358
UAL   AAL     0.891089
TMO   DHR     0.879616
XOM   COP     0.875018
DAL   AAL     0.874933
RCL   CCL     0.873982
XOM   MRO     0.871469
MRO   HAL     0.868466
UAL   DAL     0.868274
CVX   COP     0.866636
TXN   MCHP    0.866498
MCHP  AMAT    0.859095
MRO   CVX     0.859056
WFC   BAC     0.856154
JPM   BAC     0.853430
GM    F       0.850082
TXN   ADI     0.846174
XOM   HAL     0.845878
TXN   AMAT    0.844684
NVDA  AMD     0.839807
C     BAC     0.834720
HAL   COP     0.831841
SLB   MRO     0.828481
HAL   CVX     0.826548
TXN   LRCX    0.818699
dtype: float64

In [ ]:
print(s)

AAL  AAL          NaN
     ACN          NaN
     ADBE         NaN
     ADI          NaN
     ADP          NaN
               ...   
ZS   YUM     0.268633
     Z       0.485302
     ZI      0.606233
     ZM      0.582227
     ZS           NaN
Length: 24649, dtype: float64


Based on the Results, LRCX and AMAT have the highest correlation.

In [ ]:
correlations_with_AMAT = processed_corr.loc['AMAT']

In [ ]:
correlations_with_AMAT.sort_values(inplace = True, ascending = False)
print(correlations_with_AMAT[:15])

LRCX    0.934561
MCHP    0.859095
TXN     0.844684
AVGO    0.800510
AMD     0.792401
NVDA    0.791925
QCOM    0.783994
MRVL    0.772945
HPQ     0.751852
MU      0.748361
SNPS    0.719100
MA      0.707196
INTC    0.665357
MSFT    0.654514
INTU    0.646201
Name: AMAT, dtype: float64


In [ ]:
stocks = yf.download(['LRCX', 'TXN', 'AVGO', 'NVDA', 'AMAT'], start, mid)['Adj Close']
# Pull data for closes
stocks['LRCX_1d_return'] = np.log(stocks['LRCX']/stocks['LRCX'].shift(1))
stocks['AMAT_1d_return'] = np.log(stocks['AMAT']/stocks['AMAT'].shift(1))
# Calculate daily returns

[*********************100%%**********************]  5 of 5 completed


In [ ]:
print(stocks)

                  AMAT        AVGO        LRCX        NVDA         TXN  \
Date                                                                     
2022-06-06  113.377174  537.698730  505.865143  187.649734  160.313400   
2022-06-07  114.037712  548.965698  512.900635  189.048141  162.310638   
2022-06-08  110.873024  541.655579  501.818298  186.310638  158.029480   
2022-06-09  105.726669  532.898865  483.514404  180.316086  154.407730   
2022-06-10  100.442322  518.575745  464.857758  169.585846  150.776382   
...                ...         ...         ...         ...         ...   
2023-05-26  135.474426  804.043884  624.635010  389.387634  173.469864   
2023-05-30  135.982239  794.754333  626.335205  401.035431  173.725708   
2023-05-31  132.726303  799.325012  613.151489  378.269684  171.098419   
2023-06-01  134.249725  781.507385  616.293274  397.626129  172.997543   
2023-06-02  134.050598  803.321777  610.138855  393.196899  172.426819   

            LRCX_1d_return  AMAT_1d_r

In [ ]:
import numpy as np

stocks.dropna(inplace = True)


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
X = stocks[['LRCX']]
y = stocks[['AMAT']]
model = LinearRegression()
model.fit(X, y)
print(model.coef_, model.intercept_)
predicted_Y = model.predict(X)
mse = mean_squared_error(y, predicted_Y)
print(f"Mean Squared Error: {mse}")
#Simple baseline linear regression

[[0.21590049]] [4.32435139]
Mean Squared Error: 16.957509318646803


In [ ]:
from sklearn.metrics import r2_score
r2 = model.score(X, y)
print(r2)

0.9039137465939897


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
X_train = stocks[['LRCX', 'TXN', 'AVGO', 'NVDA']]
y_train = stocks[['AMAT']]
model2 = LinearRegression()
model2.fit(X, y)
print(model2.coef_, model2.intercept_)
predicted_Y = model2.predict(X)
mse = mean_squared_error(y, predicted_Y)
print(f"Mean Squared Error: {mse}")
#Simple baseline linear regression

[[ 0.10787828  0.26691432  0.07618879 -0.01038333]] [-29.11746782]
Mean Squared Error: 7.515950254153686


In [ ]:
end_stocks = yf.download(['LRCX', 'TXN', 'AVGO', 'NVDA', 'AMAT'], mid, end)['Adj Close']
X_test = end_stocks[['LRCX', 'TXN', 'AVGO', 'NVDA']]
y_test = end_stocks[['AMAT']]

[*********************100%%**********************]  5 of 5 completed


In [ ]:
pred_end_y = model.predict(X_test[['LRCX']])
mse = mean_squared_error(y_test, pred_end_y)
print("Mean Squared Error of simple linear regression is " + str(mse))

Mean Squared Error of simple linear regression is 14.986657995308635


In [ ]:
r2 = model.score(X_test[['LRCX']], y_test)
print(r2)

0.5906314194769706


In [ ]:
pred_end_y = model2.predict(X_test)
mse = mean_squared_error(y_test, pred_end_y)
print("Mean Squared Error of linear regression with multiple variables is " + str(mse))

Mean Squared Error of linear regression with multiple variables is 27.341075897922654


In [ ]:
r2 = model2.score(end_stocks[['LRCX', 'TXN', 'AVGO', 'NVDA']], end_stocks['AMAT'])
print(r2)

0.25316388524988775


In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
alpha = 0.75  # Regularization strength (adjust as needed)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
lasso_model = Lasso(alpha=alpha)
lasso_model.fit(X_train_scaled, y_train)


Lasso(alpha=0.75)

In [ ]:
train_score = lasso_model.score(X_train_scaled, y_train)
test_score = lasso_model.score(X_test_scaled, y_test)

print(f'Training R^2 Score: {train_score:.4f}')
print(f'Testing R^2 Score: {test_score:.4f}')

Training R^2 Score: 0.9533
Testing R^2 Score: 0.6516


In [ ]:
best_test_score = 0.0
for alpha in np.arange(1.0, 2.0, 0.05):
  lasso_model = Lasso(alpha=alpha)
  lasso_model.fit(X_train_scaled, y_train)
  train_score = lasso_model.score(X_train_scaled, y_train)
  test_score = lasso_model.score(X_test_scaled, y_test)
  if test_score > best_test_score:
    print("Alpha is {}".format(alpha))
    print(f'Training R^2 Score: {train_score:.4f}')
    print(f'Testing R^2 Score: {test_score:.4f}')
    best_test_score = test_score

Alpha is 1.0
Training R^2 Score: 0.9503
Testing R^2 Score: 0.7170
Alpha is 1.05
Training R^2 Score: 0.9496
Testing R^2 Score: 0.7272
Alpha is 1.1
Training R^2 Score: 0.9489
Testing R^2 Score: 0.7364
Alpha is 1.1500000000000001
Training R^2 Score: 0.9481
Testing R^2 Score: 0.7447
Alpha is 1.2000000000000002
Training R^2 Score: 0.9473
Testing R^2 Score: 0.7520
Alpha is 1.2500000000000002
Training R^2 Score: 0.9464
Testing R^2 Score: 0.7584
Alpha is 1.3000000000000003
Training R^2 Score: 0.9455
Testing R^2 Score: 0.7638
Alpha is 1.3500000000000003
Training R^2 Score: 0.9446
Testing R^2 Score: 0.7682
Alpha is 1.4000000000000004
Training R^2 Score: 0.9437
Testing R^2 Score: 0.7717
Alpha is 1.4500000000000004
Training R^2 Score: 0.9427
Testing R^2 Score: 0.7742
Alpha is 1.5000000000000004
Training R^2 Score: 0.9417
Testing R^2 Score: 0.7758
Alpha is 1.5500000000000005
Training R^2 Score: 0.9406
Testing R^2 Score: 0.7764


In [ ]:
alpha = 1.55
lasso_model = Lasso(alpha=alpha)
lasso_model.fit(X_train_scaled, y_train)
train_score = lasso_model.score(X_train_scaled, y_train)
test_score = lasso_model.score(X_test_scaled, y_test)
print(f'Training R^2 Score: {train_score:.4f}')
print(f'Testing R^2 Score: {test_score:.4f}')

Training R^2 Score: 0.9406
Testing R^2 Score: 0.7764


In [ ]:
coeffs = lasso_model.coef_
intercept = lasso_model.intercept_
print(coeffs)
print(intercept)

[5.82285727 1.82429527 4.43370863 0.        ]
[103.83096234]


In [ ]:
pred_end_y = lasso_model.predict(X_test_scaled)
mse = mean_squared_error(y_test, pred_end_y)
print("Mean Squared Error of test variables " + str(mse))

Mean Squared Error of test variables 8.185842306179227


In [ ]:
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

In [ ]:
best_test_score = 0.0
for alpha in np.arange(0.0, 2.0, 0.1):
  for l1_ratio in np.arange(0.0, 1.0, 0.05):
    elastic_net = ElasticNet(alpha=alpha, l1_ratio=l1_ratio)
    elastic_net.fit(X_train_scaled, y_train)
    train_score = elastic_net.score(X_train_scaled, y_train)
    test_score = elastic_net.score(X_test_scaled, y_test)
    if test_score > best_test_score:
      print("Alpha is {} l1_ratio is {}".format(alpha, l1_ratio))
      print(f'Training R^2 Score: {train_score:.4f}')
      print(f'Testing R^2 Score: {test_score:.4f}')
      best_test_score = test_score


<ipython-input-83-b1272d22dd1a>:5: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  elastic_net.fit(X_train_scaled, y_train)
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.357e+02, tolerance: 4.394e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
<ipython-input-83-b1272d22dd1a

Alpha is 0.0 l1_ratio is 0.0
Training R^2 Score: 0.9574
Testing R^2 Score: 0.2532
Alpha is 0.1 l1_ratio is 0.0
Training R^2 Score: 0.9532
Testing R^2 Score: 0.5729


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.275e+03, tolerance: 4.394e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.834e+03, tolerance: 4.394e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_mode

Alpha is 0.2 l1_ratio is 0.0
Training R^2 Score: 0.9492
Testing R^2 Score: 0.6277
Alpha is 0.30000000000000004 l1_ratio is 0.0
Training R^2 Score: 0.9448
Testing R^2 Score: 0.6315
Alpha is 0.30000000000000004 l1_ratio is 0.05
Training R^2 Score: 0.9453
Testing R^2 Score: 0.6338
Alpha is 0.30000000000000004 l1_ratio is 0.1
Training R^2 Score: 0.9459
Testing R^2 Score: 0.6358
Alpha is 0.30000000000000004 l1_ratio is 0.15000000000000002
Training R^2 Score: 0.9464
Testing R^2 Score: 0.6372
Alpha is 0.30000000000000004 l1_ratio is 0.2
Training R^2 Score: 0.9469
Testing R^2 Score: 0.6383
Alpha is 0.30000000000000004 l1_ratio is 0.25
Training R^2 Score: 0.9474
Testing R^2 Score: 0.6389
Alpha is 0.30000000000000004 l1_ratio is 0.30000000000000004
Training R^2 Score: 0.9480
Testing R^2 Score: 0.6390
Alpha is 0.4 l1_ratio is 0.4
Training R^2 Score: 0.9458
Testing R^2 Score: 0.6409
Alpha is 0.4 l1_ratio is 0.45
Training R^2 Score: 0.9466
Testing R^2 Score: 0.6438
Alpha is 0.4 l1_ratio is 0.5
Trai

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.852e+03, tolerance: 4.394e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.320e+03, tolerance: 4.394e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_mode

...

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.763e+03, tolerance: 4.394e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.184e+03, tolerance: 4.394e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_mode

Alpha is 0.8 l1_ratio is 0.9
Training R^2 Score: 0.9484
Testing R^2 Score: 0.6966
Alpha is 0.8 l1_ratio is 0.9500000000000001
Training R^2 Score: 0.9510
Testing R^2 Score: 0.7050
Alpha is 0.9 l1_ratio is 0.9500000000000001
Training R^2 Score: 0.9494
Testing R^2 Score: 0.7183


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.584e+03, tolerance: 4.394e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.966e+03, tolerance: 4.394e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_mode

Alpha is 1.0 l1_ratio is 0.9500000000000001
Training R^2 Score: 0.9476
Testing R^2 Score: 0.7246


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.330e+03, tolerance: 4.394e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.678e+03, tolerance: 4.394e+00 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_mode

In [ ]:
l1_ratio = 0.95
alpha = 1.0
elastic_net = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
elastic_net.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = elastic_net.predict(X_test_scaled)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
# Print the coefficients
print('Coefficients:', elastic_net.coef_)

Mean Squared Error: 10.081257346628124
Coefficients: [5.26856491 2.43129256 4.32060251 0.64131335]


In [ ]:
# Visualizing